**텍스트 정제 => 토큰화(시퀀스 변형 x) => 불용어 제거 => 텍스트 정규화 => 표제어 추출 or 어간 추출
=> 텍스트 벡터화 => 데이터 정제**

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [49]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

df = pd.read_csv('/home/kdt-admin/JY/data/drugsComTrain_raw.tsv', sep="\t", encoding='latin1')

In [5]:
df.head()

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,20-May-12,27.0
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8.0,27-Apr-10,192.0
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5.0,14-Dec-09,17.0
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8.0,03-Nov-15,10.0
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9.0,27-Nov-16,37.0


In [50]:
df.dropna(subset = ['condition'], inplace = True)
df.drop(columns = ['Unnamed: 0', 'date'], inplace = True)

In [51]:
#condition컬럼에서 이상한 값 제거
df = df[~df['condition'].str.contains("users found this comment helpful.", na = False, case = False)]


**영어, 숫자만 남겨두고 제거 => 039와 관련된 단어 삭제를 위해 숫자 남김**

In [52]:
df['review'] = df['review'].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x))

**소문자로 변환**

In [53]:
df['review'] = df['review'].str.lower()
df['drugName'] = df['drugName'].str.lower()
df['condition'] = df['condition'].str.lower()

In [10]:
df.head()

,drugName,condition,review,rating,usefulCount
0,valsartan,left ventricular dysfunction,it has no side effect i take it in combination...,9.0,27.0
1,guanfacine,adhd,my son is halfway through his fourth week of i...,8.0,192.0
2,lybrel,birth control,i used to take another oral contraceptive whic...,5.0,17.0
3,ortho evra,birth control,this is my first time using any form of birth ...,8.0,10.0
4,buprenorphine / naloxone,opiate dependence,suboxone has completely turned my life around ...,9.0,37.0


In [54]:
df['tokenized_review'] = df['review']

# **토큰화**

In [12]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from konlpy.tag import Okt
import nltk

#nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /home/kdt-
[nltk_data]    |     admin/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /home/kdt-
[nltk_data]    |     admin/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /home/kdt-admin/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /home/kdt-admin/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to /home/kdt-
[nltk_data]    |     admin/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /home/kdt-
[nltk_data]    |     admin/nl

True

In [61]:
#nltk.download('punkt')

In [55]:
def tokenize_text(text):
    tokens = word_tokenize(text)
    return tokens

df['tokenized_review'] = df['review'].apply(tokenize_text)
df['review'] = df['review'].apply(tokenize_text)

# **불용어 제거**

**토큰화만된 리뷰와 토큰화와 불용어를 제거한 리뷰 구분**

In [56]:
del_list = ['one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten', 'first', 'second', 'third', 'fourth', 'fifth', 'sixth', 'seventh', 'eighth', 'ninth',
            'week', 'weeks', 'almost', 'month', 'monthly', 'ago', 'year', 'years', 'months', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k','l', 'm', 'o', 'p', 'q', 'r', 's', 't', 'v', 'w', 'x', 'y', 'z']
stop_words_list = stopwords.words('english') + del_list
stop_words = set(stop_words_list)
words_plus = {'no', 'not'}
updated_stopwords = stop_words.difference(words_plus)

In [57]:
def remove_stopwords(text):
    filtered_words = [word for word in text if word.lower() not in updated_stopwords]
    return filtered_words


#'가 039로 깨져있음
def remove_039(tokens):
    filtered_tokens = [token for token in tokens if not re.search(r'039', token)]
    return filtered_tokens

#영어만 남기고 삭제
english_pattern = re.compile('[a-zA-Z]+')

In [16]:
df.head(15)

,drugName,condition,review,rating,usefulCount,tokenized_review
0,valsartan,left ventricular dysfunction,"[it, has, no, side, effect, i, take, it, in, c...",9.0,27.0,"[it, has, no, side, effect, i, take, it, in, c..."
1,guanfacine,adhd,"[my, son, is, halfway, through, his, fourth, w...",8.0,192.0,"[my, son, is, halfway, through, his, fourth, w..."
2,lybrel,birth control,"[i, used, to, take, another, oral, contracepti...",5.0,17.0,"[i, used, to, take, another, oral, contracepti..."
3,ortho evra,birth control,"[this, is, my, first, time, using, any, form, ...",8.0,10.0,"[this, is, my, first, time, using, any, form, ..."
4,buprenorphine / naloxone,opiate dependence,"[suboxone, has, completely, turned, my, life, ...",9.0,37.0,"[suboxone, has, completely, turned, my, life, ..."
5,cialis,benign prostatic hyperplasia,"[2nd, day, on, 5mg, started, to, work, with, r...",2.0,43.0,"[2nd, day, on, 5mg, started, to, work, with, r..."
6,levonorgestrel,emergency contraception,"[he, pulled, out, but, he, cummed, a, bit, in,...",1.0,5.0,"[he, pulled, out, but, he, cummed, a, bit, in,..."
7,aripiprazole,bipolar disorde,"[abilify, changed, my, life, there, is, hope, ...",10.0,32.0,"[abilify, changed, my, life, there, is, hope, ..."
8,keppra,epilepsy,"[i, ve, had, nothing, but, problems, with, the...",1.0,11.0,"[i, ve, had, nothing, but, problems, with, the..."
9,ethinyl estradiol / levonorgestrel,birth control,"[i, had, been, on, the, pill, for, many, years...",8.0,1.0,"[i, had, been, on, the, pill, for, many, years..."


In [58]:
df['tokenized_review'] = df['tokenized_review'].apply(remove_stopwords)
df['tokenized_review'] = df['tokenized_review'].apply(remove_039)


df['review'] = df['review'].apply(remove_039)

In [59]:
#숫자 제거
df['review'] = df['review'].apply(lambda tokens: [token for token in tokens if english_pattern.match(token)])
df['tokenized_review'] = df['tokenized_review'].apply(lambda tokens: [token for token in tokens if english_pattern.match(token)])
#df['tokenized_review'] = df['tokenized_review'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x))

**어근 추출 : Lemmatization(표제어 추출)**

In [60]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

df['tokenized_review'] = df['tokenized_review'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

In [20]:
df.head(15)

,drugName,condition,review,rating,usefulCount,tokenized_review
0,valsartan,left ventricular dysfunction,"[it, has, no, side, effect, i, take, it, in, c...",9.0,27.0,"[no, side, effect, take, combination, bystolic..."
1,guanfacine,adhd,"[my, son, is, halfway, through, his, fourth, w...",8.0,192.0,"[son, halfway, intuniv, became, concerned, beg..."
2,lybrel,birth control,"[i, used, to, take, another, oral, contracepti...",5.0,17.0,"[used, take, another, oral, contraceptive, pil..."
3,ortho evra,birth control,"[this, is, my, first, time, using, any, form, ...",8.0,10.0,"[time, using, form, birth, control, glad, went..."
4,buprenorphine / naloxone,opiate dependence,"[suboxone, has, completely, turned, my, life, ...",9.0,37.0,"[suboxone, completely, turned, life, around, f..."
5,cialis,benign prostatic hyperplasia,"[day, on, started, to, work, with, rock, hard,...",2.0,43.0,"[day, started, work, rock, hard, erection, how..."
6,levonorgestrel,emergency contraception,"[he, pulled, out, but, he, cummed, a, bit, in,...",1.0,5.0,"[pulled, cummed, bit, took, plan, hour, later,..."
7,aripiprazole,bipolar disorde,"[abilify, changed, my, life, there, is, hope, ...",10.0,32.0,"[abilify, changed, life, hope, zoloft, clonidi..."
8,keppra,epilepsy,"[i, ve, had, nothing, but, problems, with, the...",1.0,11.0,"[nothing, problem, keppera, constant, shaking,..."
9,ethinyl estradiol / levonorgestrel,birth control,"[i, had, been, on, the, pill, for, many, years...",8.0,1.0,"[pill, many, doctor, changed, rx, chateal, eff..."


# **텍스트 벡터화 or 시퀀스화 + padding**

In [61]:
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

review1 = df['review']
review2 = df['tokenized_review']

tokenizer = Tokenizer()

tokenizer.fit_on_texts(review1)
tokenizer.fit_on_texts(review2)

review1_sequences = tokenizer.texts_to_sequences(review1)
review2_sequences = tokenizer.texts_to_sequences(review2)

max_length1 = max([len(seq) for seq in review1_sequences])
df_review1_padded = pad_sequences(review1_sequences, maxlen=max_length1, padding='post')

max_length2 = max([len(seq) for seq in review2_sequences])
df_review2_padded = pad_sequences(review2_sequences, maxlen=max_length2, padding='post')

df['review'] = df_review1_padded.tolist()

df['tokenized_review'] = df_review2_padded.tolist()

In [26]:
df.head()

,drugName,condition,review,rating,usefulCount,tokenized_review
0,valsartan,left ventricular dysfunction,"[7, 58, 18, 22, 38, 1, 27, 7, 16, 750, 12, 268...",9.0,27.0,"[18, 22, 38, 27, 750, 2680, 107, 3533, 1393, 0..."
1,guanfacine,adhd,"[5, 587, 21, 3051, 335, 771, 2348, 118, 12, 30...",8.0,192.0,"[587, 3051, 3024, 358, 1123, 315, 95, 30, 25, ..."
2,lybrel,birth control,"[1, 108, 4, 27, 221, 1097, 1590, 131, 20, 35, ...",5.0,17.0,"[108, 27, 221, 1097, 1590, 35, 474, 217, 257, ..."
3,ortho evra,birth control,"[15, 21, 5, 64, 28, 143, 146, 629, 12, 97, 72,...",8.0,10.0,"[28, 143, 629, 97, 72, 542, 67, 366, 645, 746,..."
4,buprenorphine / naloxone,opiate dependence,"[828, 58, 219, 716, 5, 54, 181, 1, 34, 2135, 1...",9.0,37.0,"[828, 219, 716, 54, 181, 34, 2135, 17499, 405,..."


# **drugName, condition 처리**

In [96]:
df_1 = df.copy()

In [99]:
df_1['drugName'].nunique()

3412

In [98]:
#값들이 밀려서 condition에 날짜값같이 이상한 값들이 있어서 삭제
#drugName에 숫자만 있는 행 제거(값이 밀림)
df_1 = df_1[~df_1['condition'].str.match('\d{2}-[a-zA-Z]{3}-\d{2}', na=False)]
df_1 = df_1[~df_1['drugName'].str.match('\d+', na=False)]

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipykernel_125341/3316623514.py:2: SyntaxWarning: invalid escape sequence '\d'
  df_1 = df_1[~df_1['condition'].str.match('\d{2}-[a-zA-Z]{3}-\d{2}', na=False)]
/tmp/ipykernel_125341/3316623514.py:3: SyntaxWarning: invalid escape sequence '\d'
  df_1 = df_1[~df_1['drugName'].str.match('\d+', na=False)]


In [100]:
df_1.head()

,drugName,condition,review,rating,usefulCount,tokenized_review
0,valsartan,left ventricular dysfunction,"[7, 58, 18, 22, 38, 1, 27, 7, 16, 750, 12, 268...",9.0,27.0,"[18, 22, 38, 27, 750, 2680, 107, 3533, 1393, 0..."
1,guanfacine,adhd,"[5, 587, 21, 3051, 335, 771, 2348, 118, 12, 30...",8.0,192.0,"[587, 3051, 3024, 358, 1123, 315, 95, 30, 25, ..."
2,lybrel,birth control,"[1, 108, 4, 27, 221, 1097, 1590, 131, 20, 35, ...",5.0,17.0,"[108, 27, 221, 1097, 1590, 35, 474, 217, 257, ..."
3,ortho evra,birth control,"[15, 21, 5, 64, 28, 143, 146, 629, 12, 97, 72,...",8.0,10.0,"[28, 143, 629, 97, 72, 542, 67, 366, 645, 746,..."
4,buprenorphine / naloxone,opiate dependence,"[828, 58, 219, 716, 5, 54, 181, 1, 34, 2135, 1...",9.0,37.0,"[828, 219, 716, 54, 181, 34, 2135, 17499, 405,..."


In [101]:
df_con_encoded = pd.get_dummies(df_1['condition'])
df_drug_encoded = pd.get_dummies(df_1['drugName'])

In [102]:
df_con_encoded.head()

,abdominal distension,abnormal uterine bleeding,abortion,acetaminophen overdose,acial lipoatrophy,acial wrinkles,acne,actinic keratosis,actor ix deficiency,acute coronary syndrome,...,wegener's granulomatosis,weight loss,wilson's disease,wolff-parkinson-white syndrome,women (minoxidil),women (oxybutynin),wound cleansing,xerostomia,zen shoulde,zollinger-ellison syndrome
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [103]:
df_drug_encoded.head()

,a + d cracked skin relief,a / b otic,abacavir / dolutegravir / lamivudine,abacavir / lamivudine / zidovudine,abatacept,abilify,abilify discmelt,abilify maintena,abiraterone,abobotulinumtoxina,...,zymine,zyprexa,zyprexa intramuscular,zyprexa zydis,zyrtec,zyrtec-d,zyrtec-d 12 hour,zytiga,zyvox,zzzquil
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


# **X, y값 분할, train, test셋 분할**

In [126]:
X = pd.concat([df_1[['tokenized_review']], df_con_encoded], axis=1)
y = pd.concat([df_1[['review', 'rating', 'usefulCount']], df_drug_encoded], axis=1)

In [135]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [118]:
from tensorflow.keras import layers
from tensorflow.keras import models

model = models.Sequential()
model.add(layers.Embedding(input_dim = len(tokenizer.word_index) + 1, output_dim = 128))
model.add(layers.LSTM(64))
model.add(layers.Dense(3417, activation = 'softmax'))

2024-04-04 08:12:45.693625: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-04 08:12:45.859120: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-04 08:12:45.859322: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [119]:
from tensorflow.keras.optimizers import RMSprop
model.compile(optimizer = RMSprop(learning_rate = 0.01), loss = 'crossentropy', metrics=['acc'])

In [136]:
len(X_train)

127577

In [137]:
len(y_train)

127577

In [139]:
history = model.fit(X_train, y_train, epochs = 50, batch_size = 128, validation_split = 0.2)

ValueError: setting an array element with a sequence.

In [ ]:
X